In [167]:
import galois
import numpy as np
from numpy import binary_repr
from numpy.random import default_rng

In [168]:
length = 7
number_of_information_bits = 4
parity_check_symbols = length - number_of_information_bits
field = 3

In [169]:
reed_solomon_code = galois.ReedSolomon(7, 4)
galois_field = galois.GF(2**3)

In [170]:
parity_check_matrix = reed_solomon_code.H
print(parity_check_matrix)

[[5 7 6 3 4 2 1]
 [7 3 2 5 6 4 1]
 [6 2 7 4 5 3 1]]


In [171]:
parity_check_matrix_np = np.flip(np.array(parity_check_matrix), 1)
print(parity_check_matrix_np)

[[1 2 4 3 6 7 5]
 [1 4 6 5 2 3 7]
 [1 3 5 4 7 2 6]]


In [172]:
random_message = galois_field.Random(reed_solomon_code.k)
print(random_message)

[7 6 5 0]


In [173]:
encoded_message = reed_solomon_code.encode(random_message)
print(encoded_message)

[7 6 5 0 3 7 3]


In [174]:
encoded_message_np = np.array(encoded_message)
print(encoded_message_np)

[7 6 5 0 3 7 3]


Calculate the amount of noise to add to the channel. Will also be used to calculate the error at the end of the channel.

In [175]:
# change the symbols to binary
def symbols_to_binary(symbols, width=3):
		binary = np.zeros((len(symbols), width), dtype=int)
		for i in range(len(symbols)):
				binary[i] = np.array([int(x) for x in binary_repr(symbols[i], width=3)])
		return binary.flatten()

encoded_message_binary = symbols_to_binary(encoded_message_np, width=field)
print(encoded_message_binary)

[1 1 1 1 1 0 1 0 1 0 0 0 0 1 1 1 1 1 0 1 1]


In [176]:
# change to binary phase shift keying (BPSK) symbols
encoded_message_bpsk = 2 * encoded_message_binary - 1
print(encoded_message_bpsk)

[ 1  1  1  1  1 -1  1 -1  1 -1 -1 -1 -1  1  1  1  1  1 -1  1  1]


In [177]:
snr_range_db = np.arange(1.0, 10)
print(snr_range_db)

[1. 2. 3. 4. 5. 6. 7. 8. 9.]


In [178]:
# set the Bit Error Rate (BER) vector 
bit_error_rate = np.zeros(len(snr_range_db))
print(bit_error_rate)

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [344]:
# add noise to the code word 
def add_noise(encoded_message, snr):
    N = len(encoded_message)

    # Calculate the standard deviation of the noise from the SNR
    sigma = np.sqrt(1 / (2 * 10**(snr / 10)))

    # Generate complex Gaussian noise
    noise = sigma * (np.random.randn(N) + 1j * np.random.randn(N))

    # Add the noise to the signal
    received_signal = encoded_message + noise

    # received_signal now contains the original signal with added noise corresponding to the current SNR value
    return received_signal

received_signals = [add_noise(encoded_message_bpsk, snr) for snr in snr_range_db]
received_signals_np = np.array(received_signals)
print(received_signals_np[0])

[ 0.95811019-0.37777995j  1.57330836+0.28941233j  0.39485198+0.33433859j
  0.34749402+0.10303746j  0.26136009+0.28685293j  0.13180739-0.04681581j
  1.0212025 -0.31813141j -2.0008147 +1.0003498j   0.82575161+0.38625336j
 -0.83429991-0.77063941j -0.65785615-0.72977677j -1.79068278-0.88623732j
 -1.94910327+0.4045919j   1.32917733-0.61963799j  0.79310289-0.21729808j
 -0.2982404 +1.28283937j  1.89301561-0.52517926j  1.33963105+0.02444858j
 -1.320379  +0.4506849j  -0.0573461 -0.32490587j  1.33802611+0.55810413j]


In [345]:
# calculate the log-likelihood ratio (LLR)
def calculate_llr(received_signal, snr):
    p_y_given_0 = np.exp(-np.abs(received_signal-(-1))**2/(2*1/(10**(snr/10))))
    p_y_given_1 = np.exp(-np.abs(received_signal-1)**2/(2*1/(10**(snr/10))))

    return np.log(p_y_given_0/p_y_given_1).real

llrs = []
for i in range(len(received_signals)):
    llrs.append(calculate_llr(received_signals_np[i], snr_range_db[i]))
llrs_np = np.array(llrs)
print(llrs_np[0])

[-2.41237853 -3.96135575 -0.99417838 -0.87493811 -0.65806573 -0.33187136
 -2.57123556  5.03775295 -2.07911938  2.10064271  1.65638365  4.50867212
  4.90755127 -3.34667023 -1.99691476  0.75092482 -4.76633091 -3.37299114
  3.32451735  0.14438893 -3.36895015]


In [348]:
abs_llrs = np.abs(llrs_np)
print(abs_llrs[0])
print(abs_llrs.shape)

[2.41237853 3.96135575 0.99417838 0.87493811 0.65806573 0.33187136
 2.57123556 5.03775295 2.07911938 2.10064271 1.65638365 4.50867212
 4.90755127 3.34667023 1.99691476 0.75092482 4.76633091 3.37299114
 3.32451735 0.14438893 3.36895015]
(9, 21)


In [350]:
# calculate the binary representation of a number
def binary_representation(num, field):
    return [int(x) for x in format(num, '0' + str(field) + 'b')]

# perform binary image expansion (BIE) on H matrix
def binary_image_expansion(H_matrix, field):
    nn = len(H_matrix)
    z = np.zeros((field, nn), dtype=int)

    for j in range(nn):
        z[:, j] = binary_representation(H_matrix[j], field)

    return z

# perform binary image expansion (BIE) on H matrix
def perform_binary_image_expansion(parity_check_matrix, field):
    shape = parity_check_matrix.shape
    n = shape[1]
    # Generating GF(2^b) primitive element and bit representation
    a = galois.GF(2**field).primitive_element
    aa = a**np.arange(n)
    cx = np.concatenate((aa, aa[:field-1]))
    v = binary_image_expansion(cx, field)

    # Generating 3 by 3 square matrices for each element of the field
    sq = np.zeros((n, field, field), dtype=int)
    for i in range(n):
        sq[i,:,:] = v[:,i:i+field]

    # Generating Hb matrix
    parity_check_symbols = shape[0]  # assuming the number of parity check symbols is the number of rows in H_matrix
    Hb = np.zeros((parity_check_symbols*field, n*field), dtype=int)
    for i in range(parity_check_symbols):
        for j in range(n):
            for f in range(n):
                 if parity_check_matrix[i,j] == a**(f):
                     Hb[i*field:(i+1)*field,j*field:(j+1)*field]=sq[f,:,:]

    return Hb

parity_check_matrix_binary = perform_binary_image_expansion(parity_check_matrix_np, field)
print(parity_check_matrix_binary)

[[0 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 0 0]
 [0 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 0 0 0 0 1]
 [1 0 0 0 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 1 0]
 [0 0 1 1 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 0]
 [0 1 0 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1 1 0 0]
 [1 0 0 0 1 0 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1]
 [0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1]
 [0 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0]
 [1 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 1]]


In [355]:
def sort_indices(prob_matrix, h_matrix):
		# Get the indices that would sort the probability matrix
		sorted_indices = np.argsort(prob_matrix)
		# Sort the probability matrix and H matrix based on the sorted indices
		sorted_h_matrix = h_matrix[:, sorted_indices]  # Sort columns of H matrix based on sorted indices

		# Perform row reduction on the sorted H matrix
		Conv_gf = galois_field(sorted_h_matrix)  # Assuming GF is a valid function defined elsewhere
		RREF = Conv_gf.row_reduce(eye="left")  # Assuming row_reduce() is a valid method

		# Create a new H matrix with the original column indices after RREF
		original_indices = np.argsort(sorted_indices)  # Getting the original indices
		new_h_matrix = RREF[:, original_indices]  # Reorder columns of RREF based on original indices
		return new_h_matrix
#NewH=sort_indices(pm,BB)

sorted_indices = [sort_indices(abs_llr, parity_check_matrix_binary) for abs_llr in abs_llrs]
field_parity_check_matrix_np = np.array(sorted_indices)
print(field_parity_check_matrix_np[0])

[[0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 1 0]
 [0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 1 0 1 0 1]
 [0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1]
 [0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 0]
 [0 1 1 0 0 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0 0]
 [0 1 0 0 0 0 1 0 1 0 1 1 1 1 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0]
 [1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 1]]


In [378]:
#Sum product Algorithm
def logSPA(binary_parity_check_matrix, llr, number_of_iterations, damping_factor):
    """
    This function performs the SPA decoding algorithm in the log domain.
    It is a simplification of the SPA in the log domain.

    Arguments:
    binary_parity_check_matrix -- The Binary Parity check matrix
    LLR -- The log likelihood ratios
    number_of_iterations -- Number of iterations
    damping_factor -- Damping factor. Not used in decoding of LDPC.

    Returns:
    V -- The output message at the end of the decoding process
    Lj -- The outputted LLRs
    """

    # Initialization
    shape = binary_parity_check_matrix.shape
    t = shape[0]
    n = shape[1]

    Lrij = np.zeros((t, n))
    Lqji = binary_parity_check_matrix * np.tile(llr, (t, 1))

    Lj = np.zeros(n)
    Lext = np.zeros(n)

    for _ in range(number_of_iterations):
        # The horizontal step
        for i in range(t):
            cj = np.nonzero(binary_parity_check_matrix[i, :])[0]  # indices of nonzeros in each column
            for ij in range(len(cj)):
                lij = 1
                for in_ in range(len(cj)):
                    if in_ != ij:  # all bits except bit n
                        lij = lij * np.tanh(Lqji[i, cj[in_]] / 2)
                Lrij[i, cj[ij]] = 2 * np.arctanh(lij)  # horizontal step(CN) update

        # The vertical step
        for j in range(n):
            ri = np.nonzero(binary_parity_check_matrix[:,j])[0] # indices of nonzeros in each row

            # Finding extrinsic info
            Lext[j] = np.sum(Lrij[ri, j])

            # Finding Lj (total LLR)
            Lj[j] = llr[j] + damping_factor * np.sum(Lrij[ri, j])

    return Lj

NUMBER_OF_ITERATIONS = 1000
DAMPING_FACTOR = 0.05

spa_decoders = []
for i in range(len(llrs_np)):
    spa_decoders.append(logSPA(field_parity_check_matrix_np[i], llrs_np[i], NUMBER_OF_ITERATIONS, DAMPING_FACTOR))
spa_decoders_np = np.array(spa_decoders)
print(spa_decoders_np)


[[ -2.33459196  -3.77156947  -0.92728048  -0.97915105  -0.58843807
   -0.39236984  -2.45837345   4.92101289  -1.88850729   2.01458091
    1.59203484   4.45464643   4.75238342  -3.29760759  -1.93682308
    0.69723126  -4.72596766  -3.28912831   3.20525449   0.08254712
   -3.26236135]
 [ -3.84976566  -1.03492828  -4.36458439  -3.52952171  -5.8364068
    2.8827131   -3.40562656   2.33494168  -0.7420937    1.60546117
    0.66708885   4.28425492   3.38243352  -5.24934156  -4.62020219
    0.50849377  -3.05961643  -5.84570686   4.87335958  -4.90727026
   -4.73947071]
 [ -7.47246033  -3.53414729  -5.10657334  -7.49478505  -6.26266323
    2.95950001  -1.04272767   7.75395748  -5.59728026   3.38009246
    5.84154371   0.35255438   2.66674115  -3.70112805  -3.42739561
   -0.48808668  -2.114961    -7.06850338   4.19485783  -2.31621303
   -2.02667822]
 [ -5.90326774  -3.21111272  -7.00843405  -4.83553053  -6.57225491
    4.65636635  -7.32426078   9.99003014  -5.34697139   4.14809098
    3.72629066 

In [379]:
decoded_messages = [(spa_decoder >= 0).astype(int) for spa_decoder in spa_decoders_np]
decoded_messages_np = np.array(decoded_messages)
print(decoded_messages_np)

[[0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 1 0 0 1 1 0]
 [0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0]]


In [380]:
num_errors = [np.sum(decoded_message_np != encoded_message_binary) for decoded_message_np in decoded_messages_np]
print(num_errors)

[18, 20, 21, 21, 21, 21, 21, 21, 21]
